In [ ]:
%%writefile non_local_means_denoising.cpp
#include <opencv2/opencv.hpp>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>
#include <tuple>
#include <sys/resource.h>
#include <sys/time.h>
#include <unistd.h>

using namespace std;
using namespace cv;

chrono::high_resolution_clock::time_point start_time;

double calculate_psnr(const Mat& original, const Mat& denoised) {
    Mat diff;
    absdiff(original, denoised, diff);
    diff.convertTo(diff, CV_32F);
    diff = diff.mul(diff);

    Scalar s = sum(diff);

    double sse = s.val[0] + s.val[1] + s.val[2];
    if (sse <= 1e-10)
        return 0;
    double mse = sse / (double)(original.channels() * original.total());
    double psnr_value = 10.0 * log10((255 * 255) / mse);
    return psnr_value;
}

void print_cpu_metrics(double total_inference_time, int num_images, double total_psnr) {
    struct rusage usage;
    getrusage(RUSAGE_SELF, &usage);

    double user_time_sec = usage.ru_utime.tv_sec + usage.ru_utime.tv_usec / 1e6;
    double sys_time_sec = usage.ru_stime.tv_sec + usage.ru_stime.tv_usec / 1e6;
    double total_cpu_time_sec = user_time_sec + sys_time_sec;

    auto end = chrono::high_resolution_clock::now();
    chrono::duration<double> elapsed = end - start_time;
    double elapsed_time_sec = elapsed.count();

    double cpu_usage_percent = (total_cpu_time_sec / elapsed_time_sec) * 100.0;

    double memory_usage = usage.ru_maxrss / 1024.0;
    double avg_psnr = num_images > 0 ? total_psnr / num_images : 0;
    double avg_inference_time = num_images > 0 ? total_inference_time / num_images : 0;

    cout << "Total CPU Usage: " << cpu_usage_percent << "%" << endl;
    cout << "Total RAM Usage: " << memory_usage << " MB" << endl;
    cout << "Total Inference Time: " << total_inference_time << " ms" << endl;
    cout << "Average Inference Time per Image: " << avg_inference_time << " ms" << endl;
    cout << "Average PSNR: " << avg_psnr << " dB" << endl;
}

tuple<double, double, int> process_folder(const string& input_folder, const string& output_folder) {
    filesystem::create_directory(output_folder);

    double folder_inference_time = 0;
    double folder_psnr_sum = 0;
    int num_images = 0;

    for (const auto& entry : filesystem::directory_iterator(input_folder)) {
        if (entry.path().extension() == ".png" || entry.path().extension() == ".jpg" || entry.path().extension() == ".jpeg") {
            string filename = entry.path().string();
            Mat image = imread(filename);

            if (image.empty()) continue;

            if (image.rows > 720 || image.cols > 1280) {
                cerr << "Input image size exceeds 1280x720 resolution: " << filename << endl;
                continue;
            }

            auto start = chrono::high_resolution_clock::now();
            Mat denoised_image;
            if (image.channels() == 1) {
                fastNlMeansDenoising(image, denoised_image);
            } else {
                fastNlMeansDenoisingColored(image, denoised_image);
            }
            auto end = chrono::high_resolution_clock::now();
            double inference_time = chrono::duration_cast<chrono::milliseconds>(end - start).count();

            double psnr_value = calculate_psnr(image, denoised_image);
            folder_psnr_sum += psnr_value;
            folder_inference_time += inference_time;
            num_images++;

            string output_file_path = output_folder + "/denoised_" + entry.path().filename().string();
            imwrite(output_file_path, denoised_image);
        }
    }

    return make_tuple(folder_inference_time, folder_psnr_sum, num_images);
}

int main() {
    start_time = chrono::high_resolution_clock::now();
    double total_inference_time = 0;
    double total_psnr = 0;
    int total_images = 0;

    vector<string> folders = {"noisy5", "noisy15", "noisy35"};
    for (const auto& folder : folders) {
        string input_folder = "/content/drive/My Drive/dataset/" + folder;
        string output_folder = "/content/drive/My Drive/dataset/denoised" + folder.substr(5);

        auto [inference_time, psnr_sum, num_images] = process_folder(input_folder, output_folder);
        total_inference_time += inference_time;
        total_psnr += psnr_sum;
        total_images += num_images;
    }

    print_cpu_metrics(total_inference_time, total_images, total_psnr);

    return 0;
}

In [ ]:
!g++ -std=c++17 -o non_local_means_denoising non_local_means_denoising.cpp `pkg-config --cflags --libs opencv4`

!./non_local_means_denoising